**Train Test and Cross Validation**

In [2]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier


In [3]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


In [4]:
# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [5]:
# Decision Tree Classifier setup
dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# Hyperparameter grid for GridSearchCV
param_grid = {
    'max_depth': [3, 4, 5, 6, 7],  # Different depths to test
    'min_samples_split': [2, 5, 10],
}

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV setup with F1-score as the scoring parameter
grid_search = GridSearchCV(dt, param_grid, scoring='f1', cv=kf, n_jobs=-1, verbose=1)


In [6]:
# Fit GridSearchCV on the training data
grid_search.fit(X_scaled, y)

# Best Decision Tree model from GridSearchCV
best_dt = grid_search.best_estimator_


Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [7]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)


In [8]:
# Predictions on validation data
y_pred_valid = best_dt.predict(X_valid_scaled)


In [9]:
# Print classification report on validation data
print(f"📌 DT - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 DT - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 DT - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.65      0.71       178
           1       0.71      0.83      0.77       186

    accuracy                           0.74       364
   macro avg       0.75      0.74      0.74       364
weighted avg       0.75      0.74      0.74       364

🟦 DT - Validation Confusion Matrix:
[[115  63]
 [ 31 155]]


In [10]:
import pandas as pd
from sklearn.metrics import classification_report

# Assuming you already have y_valid (true labels) and y_pred_valid (predicted labels)
# Example classification report for AdaBoost (AB)
report = classification_report(y_valid, y_pred_valid, output_dict=True)

# Extract weighted scores
weighted_precision = report['weighted avg']['precision']
weighted_recall = report['weighted avg']['recall']
weighted_f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

# Create a dictionary with the scores
ab_results = {
    'Model': 'Decision Tree(DT)',
    'Precision': weighted_precision,
    'Recall': weighted_recall,
    'F1-Score': weighted_f1,
    'Accuracy': accuracy
}

# Convert the dictionary to a DataFrame
ab_df = pd.DataFrame([ab_results])

# Create a DataFrame to store results (if not already created)
results_df = pd.read_csv("/workspaces/Final-Year-Project/Results/model_results.csv")

# Concatenate the new row to the DataFrame
results_df = pd.concat([results_df, ab_df], ignore_index=True)

# Print the DataFrame to confirm it's added
print(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv("/workspaces/Final-Year-Project/Results/model_results.csv", index=False)


                               Model  Precision    Recall  F1-Score  Accuracy
0                      AdaBoost (AB)   0.767212  0.752747  0.748389  0.752747
1           Logistic Regression (LR)   0.753372  0.750000  0.748645  0.750000
2                    Naive Bayes(LR)   0.763240  0.755495  0.752966  0.755495
3  K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527
4                  Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758


In [11]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Macro F1-Score on Validation Data: {validation_f1_macro}")


🏆 Macro F1-Score on Validation Data: 0.738601637941572
